In [1]:
import pandas as pd
from nltk.tokenize import sent_tokenize
import re

In [2]:
df = pd.read_csv('../value_identification/generations/generation_responses/acl_gemini.csv')
df = df[df.response.notna()]

In [3]:
df

,ACL_ID,Prompt,response
0,W12-2037,I have the following research paper:\nThe grow...,"- ""Experimental results using Wikipedia as the..."
1,W12-2037,I have the following research paper:\nThe grow...,This research paper does not discuss the subje...
2,P07-1058,I have the following research paper:\nObtainin...,I cannot find any excerpts in the provided tex...
3,P07-1058,I have the following research paper:\nObtainin...,This research paper does not mention ease of i...
4,P07-1058,I have the following research paper:\nObtainin...,The provided research paper does not mention f...
...,...,...,...
5833,W16-4804,I have the following research paper:\nThis pap...,**There is no mention of reproducibility in th...
5834,W12-2037,I have the following research paper:\nThe grow...,This research paper does not explicitly mentio...
5835,W12-2037,I have the following research paper:\nThe grow...,The provided text does not explicitly mention ...
5836,W12-2037,I have the following research paper:\nThe grow...,There is no mention of fairness in the provide...


In [4]:
def sent_tokens(text):
    sents = []
    for line in text.split('\n'):
        if text != '':
            sents.extend(sent_tokenize(line))
    return sents

In [5]:
df['response_sents'] = df.response.map(sent_tokens)

In [6]:
def clean_sents(sent_list):
    return [re.sub(r'^\d+\.', '', s.lstrip('- ').replace('*', '')).strip().strip('"') for s in sent_list]

In [7]:
df['cleaned_response_sents'] = df.response_sents.map(clean_sents)

In [8]:
def filter_sents(row):
    return [s for s in row.cleaned_response_sents if s in row.Prompt]

In [9]:
df['filtered_sents'] = df.apply(filter_sents, axis=1)

In [10]:
(df.filtered_sents.map(len) > 0).mean()

0.3663194444444444

In [11]:
df.response = df.filtered_sents.map(lambda l: ' '.join(l))

In [12]:
df['value'] = df.Prompt.map(lambda p: p.split('\n\n')[-1].split(', which refers to')[0][53:])

In [13]:
subset = df.loc[:, ['ACL_ID', 'value', 'response']]

In [14]:
subset.columns = ['acl_id', 'value', 'response']

In [15]:
subset

,acl_id,value,response
0,W12-2037,performance,Experimental results using Wikipedia as the co...
1,W12-2037,reproducibility,
2,P07-1058,building on past work,
3,P07-1058,ease of implementation,
4,P07-1058,fairness,
...,...,...,...
5833,W16-4804,reproducibility,
5834,W12-2037,building on past work,
5835,W12-2037,ease of implementation,
5836,W12-2037,fairness,


In [17]:
subset.to_csv('cleaned_acl_gemini.csv', index=False)